In [1]:
import pandas as pd
import numpy as np

# 1. Load data


In [2]:
data_path = "ChallengeXHEC23022024.xlsx"
dict_df_sheets = pd.read_excel(data_path, sheet_name=None)
df_hist_jan = dict_df_sheets["JAN24"].copy()
df_clients = dict_df_sheets["clients"].copy()
df_inter = dict_df_sheets["intervenants"].copy()

# 2. Data exploration preprocessing


In [6]:
df_hist_jan.rename(
    {
        "ID Client": "id_client",
        "ID Intervenant": "id_inter",
        "Date": "date",
        "Heure de début": "start",
        "Heure de fin": "end",
        "Prestation": "obj",
    },
    axis=1,
    inplace=True,
)

df_clients.rename(
    {"ID Client": "id_client", "Latitude": "lat_client", "Longitude": "lon_client"},
    axis=1,
    inplace=True,
)

df_inter.rename(
    {
        "ID Intervenant": "id_inter",
        "Latitude": "lat_inter",
        "Longitude": "lon_inter",
        "Compétences": "skills",
        "Permis": "license",
        "Véhicule personnel": "car",
        "Dispo / Indispo": "free",
    },
    axis=1,
    inplace=True,
)

In [7]:
df_hist_jan["date"] = pd.to_datetime(df_hist_jan["date"])
df_hist_jan["start"] = pd.to_datetime(df_hist_jan["date"])
df_hist_jan["date"] = pd.to_datetime(df_hist_jan["date"])

# 5. Modeling with Google OR Tools


In [10]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from functools import partial

### 5.C.1. With initial locations, time windows, bike/car time travel, task execution time and skills with REAL DATA


In [28]:
c_c_dist = pd.read_csv("client_client_distance.csv")
c_l_need = pd.read_csv("client_loc_need.csv")
c_n_dist = pd.read_csv("client_nurse_distance.csv")

list_clients_1 = c_c_dist["ID Client 1"].unique()
list_clients_2 = c_c_dist["ID Client 2"].unique()
full_client_list = list(set([*list_clients_1, *list_clients_2]))
nb_full_client = len(full_client_list)

In [29]:
client_id_to_idx = {}
client_idx_to_id = {}
for k in range(len(full_client_list)):
    client_id_to_idx[full_client_list[k]] = k
    client_idx_to_id[k] = full_client_list[k]

In [30]:
def convert_duration(str_duration):
    if "hour" in str_duration:
        list = str_duration.split()
        nb_minutes = int(list[0]) * 60 + int(list[2])
        return nb_minutes
    else:
        list = str_duration.split()
        nb_minutes = int(list[0])
        return nb_minutes

In [31]:
def define_client_time_matrix(
    df, vehicle_type, client_id_to_idx=client_id_to_idx, out_value=4000
):
    list_clients_1 = df["ID Client 1"].unique()
    list_clients_2 = df["ID Client 2"].unique()
    full_list = list(set([*list_clients_1, *list_clients_2]))

    matrix = [[0 for i in range(len(full_list))] for k in range(len(full_list))]
    col_type = "Duration_" + vehicle_type
    for client_1_id in full_list:
        client_1_idx = client_id_to_idx[client_1_id]
        for client_2_id in full_list:
            client_2_idx = client_id_to_idx[client_2_id]
            if client_1_id != client_2_id:
                try:
                    travel_time = convert_duration(
                        df.loc[
                            (df["ID Client 1"] == client_1_id)
                            & (df["ID Client 2"] == client_2_id),
                            col_type,
                        ].values[0]
                    )
                    matrix[client_1_idx][client_2_idx] = travel_time
                except IndexError:
                    try:
                        travel_time = convert_duration(
                            df.loc[
                                (df["ID Client 1"] == client_2_id)
                                & (df["ID Client 2"] == client_1_id),
                                col_type,
                            ].values[0]
                        )
                        matrix[client_1_idx][client_2_idx] = travel_time
                    except IndexError:
                        matrix[client_1_idx][client_2_idx] = out_value

    return np.array(matrix)

In [32]:
client_time_matrix_car = define_client_time_matrix(c_c_dist, vehicle_type="Car")
client_time_matrix_bike = define_client_time_matrix(c_c_dist, vehicle_type="Bike")

In [33]:
list_inter_columns = c_n_dist.columns[2:]
full_inter_list = list(set([int(x.split(sep="_")[2]) for x in list_inter_columns]))

inter_id_to_idx = {}
inter_idx_to_id = {}
for k in range(len(full_inter_list)):
    inter_id_to_idx[full_inter_list[k]] = k
    inter_idx_to_id[k] = full_inter_list[k]

In [34]:
def define_inter_time_matrix(
    df,
    full_client_list,
    inter_id_to_idx=inter_id_to_idx,
    client_id_to_idx=client_id_to_idx,
    vehicle_type="Car",
    out_value=4000,
):
    list_inter_columns = df.columns[2:]
    full_inter_list = list(set([int(x.split(sep="_")[2]) for x in list_inter_columns]))

    matrix = [
        [0 for i in range(len(full_client_list))] for k in range(len(full_inter_list))
    ]

    for inter_id in full_inter_list:
        vehicle_str = "_Driving" if vehicle_type == "Car" else "_Bicycling"
        col_name = "Duration_Intervenant_" + str(inter_id) + vehicle_str
        inter_idx = inter_id_to_idx[inter_id]

        for client_id in full_client_list:
            client_idx = client_id_to_idx[client_id]
            try:
                travel_time = convert_duration(
                    df.loc[df["ID Client"] == client_id, col_name].values[0]
                )
                matrix[inter_idx][client_idx] = travel_time
            except IndexError:
                matrix[inter_idx][client_idx] = out_value

    return np.array(matrix)

In [35]:
inter_time_matrix_car = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Car"
)
inter_time_matrix_bike = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Bike"
)

In [36]:
def create_full_time_matrix(client_time_matrix, inter_time_matrix, out_value=4000):
    nb_client = client_time_matrix.shape[0]
    nb_inter = inter_time_matrix.shape[0]

    matrix = [
        [out_value for i in range(nb_client + nb_inter)]
        for k in range(nb_client + nb_inter)
    ]

    for i in range(nb_client):
        for j in range(nb_client):
            matrix[i][j] = client_time_matrix[i][j]

    for i in range(nb_inter):
        for j in range(nb_client):
            matrix[i + nb_client][j] = inter_time_matrix[i][j]
            matrix[j][i + nb_client] = matrix[i + nb_client][j]

    return np.array(matrix)

In [37]:
full_time_matrix_car = create_full_time_matrix(
    client_time_matrix_car, inter_time_matrix_car
)
full_time_matrix_bike = create_full_time_matrix(
    client_time_matrix_bike, inter_time_matrix_bike
)

In [47]:
list_skills = c_l_need["Prestation"].unique()

condition = c_l_need["Date"] == "2024-01-01"
c_l_test = c_l_need[condition]
c_l_test.drop(columns=["Unnamed: 0", "Date", "Latitude", "Longitude"], inplace=True)
# c_l_test = c_l_test[:70]
c_l_test.head()

/var/folders/95/2w4zxm7j571dt7gznpftznw80000gn/T/ipykernel_79695/2604372886.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["Unnamed: 0", "Date", "Latitude", "Longitude"], inplace=True)


,ID Client,Prestation,Durée,tranche_horaire
0,559475456,REPAS,30.0,"[7, 9]"
1,559277088,TOILETTE,45.0,"[7, 11]"
2,87852633,TOILETTE,45.0,"[7, 11]"
3,243033408,TOILETTE,30.0,"[7, 11]"
4,814940942,TOILETTE,95.0,"[7, 11]"


# 6. Data preparation for scenarios


In [53]:
data_path = "ChallengeXHEC23022024.xlsx"
dict_df_sheets = pd.read_excel(data_path, sheet_name=None)
df_hist_jan = dict_df_sheets["JAN24"].copy()
df_clients = dict_df_sheets["clients"].copy()
df_inter = dict_df_sheets["intervenants"].copy()

In [54]:
client_locations = [
    (df_clients["Latitude"][k], df_clients["Longitude"][k])
    for k in range(len(df_clients))
]

inter_locations = [
    (df_inter["Latitude"][k], df_inter["Longitude"][k]) for k in range(len(df_inter))
]

In [55]:
client_id_to_location = {}
for k in range(len(df_clients)):
    client_id_to_location[int(df_clients["ID Client"][k])] = client_locations[k]

inter_id_to_location = {}
for k in range(len(df_inter)):
    inter_id_to_location[int(df_inter["ID Intervenant"][k])] = inter_locations[k]

# 6.0. Travel time estimator


In [56]:
from sklearn.ensemble import RandomForestRegressor

In [57]:
nb_clients = len(full_client_list)
nb_inter = len(full_inter_list)
nb_loc = len(full_time_matrix_car)

travel_time_data = []

for i in range(nb_loc - 1):
    for j in range(i + 1, nb_loc):
        row = []
        if i < nb_clients:
            client_id_1 = client_idx_to_id[i]
            row.append(client_id_to_location[client_id_1][0])
            row.append(client_id_to_location[client_id_1][1])

            if j < nb_clients:
                client_id_2 = client_idx_to_id[j]
                row.append(client_id_to_location[client_id_2][0])
                row.append(client_id_to_location[client_id_2][1])
            else:
                inter_id_2 = inter_idx_to_id[j - nb_clients]
                row.append(inter_id_to_location[inter_id_2][0])
                row.append(inter_id_to_location[inter_id_2][1])

        else:
            inter_id = inter_idx_to_id[i - nb_clients]
            row.append(inter_id_to_location[inter_id][0])
            row.append(inter_id_to_location[inter_id][1])

            if j < nb_clients:
                client_id_2 = client_idx_to_id[j]
                row.append(client_id_to_location[client_id_2][0])
                row.append(client_id_to_location[client_id_2][1])
            else:
                inter_id_2 = inter_idx_to_id[j - nb_clients]
                row.append(inter_id_to_location[inter_id_2][0])
                row.append(inter_id_to_location[inter_id_2][1])

        row.append(full_time_matrix_car[i][j])
        row.append(full_time_matrix_bike[i][j])

        travel_time_data.append(row)

travel_time_df = pd.DataFrame(
    travel_time_data, columns=["x_1", "y_1", "x_2", "y_2", "car", "bike"]
)
travel_time_df

,x_1,y_1,x_2,y_2,car,bike
0,48.732218,1.363744,48.705077,1.333307,7,14
1,48.732218,1.363744,48.638658,1.517034,24,63
2,48.732218,1.363744,48.787278,1.441202,16,38
3,48.732218,1.363744,48.768307,1.403410,10,24
4,48.732218,1.363744,48.725093,1.427367,11,25
...,...,...,...,...,...,...
9586,48.758099,1.210611,48.689927,1.351054,4000,4000
9587,48.758099,1.210611,48.763226,1.241120,4000,4000
9588,48.729206,1.371985,48.689927,1.351054,4000,4000
9589,48.729206,1.371985,48.763226,1.241120,4000,4000


In [58]:
condition = travel_time_df["car"] != 4000
car_travel_time_df = travel_time_df[condition].copy()
X_car = np.array(car_travel_time_df.drop(columns=["car", "bike"]))
y_car = np.array(car_travel_time_df["car"])

car_time_travel_estimator = RandomForestRegressor()
car_time_travel_estimator.fit(X_car, y_car)

RandomForestRegressor()

In [59]:
condition = travel_time_df["bike"] != 4000
bike_travel_time_df = travel_time_df[condition].copy()
X_bike = np.array(bike_travel_time_df.drop(columns=["car", "bike"]))
y_bike = np.array(bike_travel_time_df["bike"])

bike_time_travel_estimator = RandomForestRegressor()
bike_time_travel_estimator.fit(X_bike, y_bike)

RandomForestRegressor()

## 6.1. Add new client


The goal is to create artificially a new client the following way:

- Get a random ('Prestation', 'Durée', 'Tranche horaire') from an existing client;
- Get a random location

From the random location, infere the travel time to each other node


In [62]:
def add_clients_in_time_matrix(
    client_synthetic_locations,
    time_matrix_car,
    time_matrix_bike,
    df_og_need,
    list_available_inter,
):
    nb_og_clients = len(df_og_need)
    len_new_matrix = (
        nb_og_clients + len(client_synthetic_locations) + len(list_available_inter)
    )
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    nb_synth_clients = len(client_synthetic_locations)

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_synth_clients:
                    # client_id_1 = client_idx_to_id[i]
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    new_car_matrix[i][j] = time_matrix_car[i][j - nb_synth_clients]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j - nb_synth_clients]
            elif nb_og_clients <= i < nb_og_clients + nb_synth_clients:
                if j < nb_og_clients + nb_synth_clients:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    # inter_id_1 = inter_idx_to_id[j - nb_og_clients - nb_synth_clients]
                    # print(i)
                    # print(j)
                    # print(j - nb_og_clients - nb_synth_clients)
                    # print("\n")
                    inter_id_1 = list_available_inter[
                        j - nb_og_clients - nb_synth_clients
                    ]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = time_matrix_car[i - nb_synth_clients][
                    j - nb_synth_clients
                ]
                new_bike_matrix[i][j] = time_matrix_bike[i - nb_synth_clients][
                    j - nb_synth_clients
                ]

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

## 6.2. Add new inter


The goal is to create artificially a new inter the following way:

- Get a random ('Skills', 'Disponibility', 'Car') from existing inter (can be different ones);
- Get a random location

From the random location, infere the travel time to each other node


In [63]:
def add_clients_and_inter_in_time_matrix(
    client_synthetic_locations,
    inter_synthetic_locations,
    time_matrix_car,
    time_matrix_bike,
    df_og_need,
    list_available_inter,
):
    nb_og_clients = len(df_og_need)
    nb_og_inter = len(list_available_inter)
    len_new_matrix = (
        len(client_synthetic_locations)
        + len(inter_synthetic_locations)
        + len(time_matrix_car)
    )
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    nb_synth_clients = len(client_synthetic_locations)
    nb_synth_inter = len(inter_synthetic_locations)

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_synth_clients:
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                elif (
                    nb_og_clients + nb_synth_clients
                    <= j
                    < nb_og_clients + nb_synth_clients + nb_og_inter
                ):
                    new_car_matrix[i][j] = time_matrix_car[i][j - nb_synth_clients]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j - nb_synth_clients]
                else:
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif nb_og_clients <= i < nb_og_clients + nb_synth_clients:
                if j < nb_og_clients + nb_synth_clients:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                elif (
                    nb_og_clients + nb_synth_clients
                    <= j
                    < nb_og_clients + nb_synth_clients + nb_og_inter
                ):
                    # inter_id_1 = inter_idx_to_id[j - nb_og_clients - nb_synth_clients]
                    inter_id_1 = list_available_inter[
                        j - nb_og_clients - nb_synth_clients
                    ]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif (
                nb_og_clients + nb_synth_clients
                <= i
                < nb_og_clients + nb_synth_clients + nb_og_inter
            ):
                if j < nb_og_clients + nb_synth_clients + nb_og_inter:
                    new_car_matrix[i][j] = time_matrix_car[i - nb_synth_clients][
                        j - nb_synth_clients
                    ]
                    new_bike_matrix[i][j] = time_matrix_bike[i - nb_synth_clients][
                        j - nb_synth_clients
                    ]
                else:
                    # inter_id_1 = inter_idx_to_id[i - nb_synth_clients - nb_og_clients]
                    inter_id_1 = list_available_inter[
                        i - nb_og_clients - nb_synth_clients
                    ]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = round(
                    car_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )
                new_bike_matrix[i][j] = round(
                    bike_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

In [64]:
def add_inter_in_time_matrix(
    inter_synthetic_locations,
    time_matrix_car,
    time_matrix_bike,
    df_og_need,
    list_available_inter,
):
    nb_og_clients = len(df_og_need)
    nb_og_inter = len(list_available_inter)
    len_new_matrix = +len(inter_synthetic_locations) + len(time_matrix_car)
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_og_inter:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                else:
                    # client_id_1 = client_idx_to_id[i]
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif nb_og_clients <= i < nb_og_clients + nb_og_inter:
                if j < nb_og_clients + nb_og_inter:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                else:
                    # inter_id_1 = inter_idx_to_id[i - nb_og_clients]
                    inter_id_1 = list_available_inter[i - nb_og_clients]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = round(
                    car_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )
                new_bike_matrix[i][j] = round(
                    bike_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

## 6.3. Add actors refined


In [68]:
def sample_random_loc(nb_synth_loc, list_og_locations):
    og_x_coords, og_y_coords = zip(*list_og_locations)

    # Calculate mean and standard deviation for x and y
    og_mean_x, og_std_x = np.mean(og_x_coords), np.std(og_x_coords)
    og_mean_y, og_std_y = np.mean(og_y_coords), np.std(og_y_coords)

    # Generate synthetic locations
    list_synth_locations = [
        (
            np.random.normal(og_mean_x, og_std_x),
            np.random.normal(og_mean_y, og_std_y),
        )
        for _ in range(nb_synth_loc)
    ]

    return list_synth_locations

In [69]:
def add_actors(
    df_og_needs,
    df_og_inter,
    list_available_inter,
    time_matrix_car,
    time_matrix_bike,
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_locations=client_locations,
    inter_locations=inter_locations,
):
    # nb_og_clients = len(df_og_needs)
    # nb_og_inter = len(list_available_inter)
    if nb_synth_inter == 0:
        list_synth_client_loc = sample_random_loc(nb_synth_clients, client_locations)
        new_car_time_matrix, new_bike_time_matrix = add_clients_in_time_matrix(
            list_synth_client_loc,
            time_matrix_car,
            time_matrix_bike,
            df_og_needs,
            list_available_inter,
        )

        synth_client_needs_df = df_og_needs[
            ["Prestation", "Durée", "tranche_horaire"]
        ].sample(n=nb_synth_clients, replace=True)
        synth_client_needs_df.reset_index(drop=True, inplace=True)

        synth_inter_df = pd.DataFrame()

    elif nb_synth_clients == 0:
        list_synth_inter_loc = sample_random_loc(nb_synth_inter, inter_locations)
        new_car_time_matrix, new_bike_time_matrix = add_inter_in_time_matrix(
            list_synth_inter_loc,
            time_matrix_car,
            time_matrix_bike,
            df_og_needs,
            list_available_inter,
        )

        synth_client_needs_df = pd.DataFrame()

        synth_inter_df = df_og_inter[
            ["Compétences", "Permis", "Véhicule personnel", "Dispo / Indispo"]
        ].sample(n=nb_synth_inter, replace=True)
        synth_inter_df.reset_index(drop=True, inplace=True)

    else:
        list_synth_client_loc = sample_random_loc(nb_synth_clients, client_locations)
        list_synth_inter_loc = sample_random_loc(nb_synth_inter, inter_locations)
        new_car_time_matrix, new_bike_time_matrix = (
            add_clients_and_inter_in_time_matrix(
                list_synth_client_loc,
                list_synth_inter_loc,
                time_matrix_car,
                time_matrix_bike,
                df_og_needs,
                list_available_inter,
            )
        )

        synth_client_needs_df = df_og_needs[
            ["Prestation", "Durée", "tranche_horaire"]
        ].sample(n=nb_synth_clients, replace=True)
        synth_client_needs_df.reset_index(drop=True, inplace=True)

        synth_inter_df = df_og_inter[
            ["Compétences", "Permis", "Véhicule personnel", "Dispo / Indispo"]
        ].sample(n=nb_synth_inter, replace=True)
        synth_inter_df.reset_index(drop=True, inplace=True)

    return (
        new_car_time_matrix,
        new_bike_time_matrix,
        synth_client_needs_df,
        synth_inter_df,
    )

In [70]:
def create_real_data_model_with_synth(
    df_og_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
    df_inter=df_inter,
):
    nb_needs = len(df_og_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["og_time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["og_time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["service_times"] = [
        0 for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]
    data["time_windows"] = [
        (0, 1440)
        for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    data["num_vehicles"] = nb_inter + nb_synth_inter
    data["is_car"] = [
        1 for i in range(nb_inter + nb_synth_inter)
    ]  # For the moment every inter has a car
    data["starts"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]
    data["ends"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]

    data["vehicle_skills"] = [
        list_skills for i in range(nb_inter + nb_synth_inter)
    ]  # For the moment every inter has all skills possible
    data["node_requirements"] = [
        None for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    # Create original time matrix
    for i in range(nb_needs):
        need_id_client = df_og_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_og_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["og_time_matrix_car"][i][j] = travel_time_car
            data["og_time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["og_time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["og_time_matrix_car"][nb_needs + j][i] = data["og_time_matrix_car"][i][
                nb_needs + j
            ]
            data["og_time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["og_time_matrix_bike"][nb_needs + j][i] = data["og_time_matrix_bike"][
                i
            ][nb_needs + j]
    for i in range(nb_needs + nb_inter):
        data["og_time_matrix_car"][i][i] = 0
        data["og_time_matrix_bike"][i][i] = 0

    # Create new time matrix from orginial adding synthetic travel times
    (
        data["time_matrix_car"],
        data["time_matrix_bike"],
        synth_client_needs_df,
        synth_inter_df,
    ) = add_actors(
        df_og_need,
        df_inter,
        list_available_inter,
        data["og_time_matrix_car"],
        data["og_time_matrix_bike"],
        nb_synth_clients,
        nb_synth_inter,
        client_locations,
        inter_locations,
    )

    for i in range(nb_needs + nb_synth_clients):
        if i < nb_needs:
            service_time = int(df_og_need["Durée"][i])
            data["service_times"][i] = service_time

            time_window_temp = (
                df_og_need["tranche_horaire"][i]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = df_og_need["Prestation"][i]
            data["node_requirements"][i] = skills_required
        else:
            service_time = int(synth_client_needs_df["Durée"][i - nb_needs])
            data["service_times"][i] = service_time

            time_window_temp = (
                synth_client_needs_df["tranche_horaire"][i - nb_needs]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = synth_client_needs_df["Prestation"][i - nb_needs]
            data["node_requirements"][i] = skills_required

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    data["workload_per_node"] = [1 for i in range(nb_needs + nb_synth_clients)].append(
        [0 for k in range(nb_inter + nb_synth_inter)]
    )  # Useless for now but could be used to differ workload from each node
    data["workload_capacity"] = [5 for i in range(nb_inter + nb_synth_inter)]

    return data

## 6.4. Refined printing result function


In [71]:
def eq_co2_car(travel_time):
    # 10km = 1.9kg CO2
    speed = 0.5  # km/min
    return 1.9 * travel_time * speed / 10


def eq_co2_elec_bike(travel_time):
    # 10km = 20g CO2
    speed = 1 / 3  # km/min
    return 0.02 * travel_time * speed / 10

In [72]:
def print_detailed_solution_real_data(
    data, manager, routing, solution, list_available_inter
):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")

    nb_cars_used = 0
    nb_bikes_used = 0
    nb_inter_used = 0

    nb_break_inf = 0
    nb_break_sup = 0
    nb_visits = 0

    avg_length_day = 0
    list_length_day = []

    overall_total_car_commuting_time = 0
    overall_total_bike_commuting_time = 0
    overall_total_commuting_time = 0  # Initialize overall commuting time

    # Display dropped nodes.
    dropped_nodes = "Dropped nodes:"
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += f" {manager.IndexToNode(node)};"
    print(dropped_nodes)

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        try:
            inter_id = list_available_inter[vehicle_id]
        except IndexError:
            inter_id = "*synthetic*"
        vehicle_type_str = "Car" if data["is_car"][vehicle_id] else "Bike"
        print(
            f"Route for vehicle {inter_id} (Type: {vehicle_type_str}; Skills: {vehicle_skills}):"
        )

        vehicle_commuting_time = 0  # Initialize commuting time for the current vehicle
        nb_steps = 0
        start_day = 0
        end_day = 0
        nb_break_inf_veh = 0
        nb_break_sup_veh = 0

        # print(f"Route for vehicle {inter_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            if nb_steps == 1:
                if data["is_car"][vehicle_id]:
                    nb_cars_used += 1
                else:
                    nb_bikes_used += 1
                nb_inter_used += 1

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            # Initialization
            departure_time = end_of_service_time

            if data["is_car"][vehicle_id]:
                next_travel_time = data["time_matrix_car"][manager.IndexToNode(index)][
                    next_node
                ]
            else:
                next_travel_time = data["time_matrix_bike"][manager.IndexToNode(index)][
                    next_node
                ]
            vehicle_commuting_time += next_travel_time
            if data["is_car"][vehicle_id]:
                overall_total_car_commuting_time += next_travel_time
            else:
                overall_total_bike_commuting_time += next_travel_time
            overall_total_commuting_time += next_travel_time

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if nb_steps > 0:
                if break_time > 30:
                    nb_break_sup += 1
                    nb_break_sup_veh += 1
                    nb_visits += 1
                else:
                    nb_visits += 1
                    if next_index != routing.End(vehicle_id):
                        nb_break_inf += 1
                        nb_break_inf_veh += 1

            if nb_steps == 0:
                start_day = departure_time

            nb_steps += 1

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            arrival_time_str = (
                str(arrival_time // 60)
                + "h"
                + (
                    str(arrival_time % 60)
                    if arrival_time % 60 >= 10
                    else "0" + str(arrival_time % 60)
                )
            )
            end_of_service_time_str = (
                str(end_of_service_time // 60)
                + "h"
                + (
                    str(end_of_service_time % 60)
                    if end_of_service_time % 60 >= 10
                    else "0" + str(end_of_service_time % 60)
                )
            )
            break_time_str = (
                str(break_time // 60)
                + "h"
                + (
                    str(break_time % 60)
                    if break_time % 60 >= 10
                    else "0" + str(break_time % 60)
                )
            )
            departure_time_str = (
                str(departure_time // 60)
                + "h"
                + (
                    str(departure_time % 60)
                    if departure_time % 60 >= 10
                    else "0" + str(departure_time % 60)
                )
            )

            # Print detailed timing for current location
            print(
                f"* Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time_str}, "
                f"End of Service Time (Skills required: {skills_required}): {end_of_service_time_str}, Break: {break_time_str}, "
                f"Departure: {departure_time_str}) -> "
            )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        end_day = final_arrival_time
        final_arrival_time_str = (
            str(final_arrival_time // 60)
            + "h"
            + (
                str(final_arrival_time % 60)
                if final_arrival_time % 60 >= 10
                else "0" + str(final_arrival_time % 60)
            )
        )
        print(
            f"* Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time_str})"
        )
        print(
            f"Number of small breaks (<30min): {nb_break_inf_veh}; Number of big breaks (>30min): {nb_break_sup_veh}"
        )
        print(
            f"Total commuting time for vehicle {inter_id}: {vehicle_commuting_time} minutes"
        )

        if end_day - start_day > 1:  # Superior than 1 minute
            list_length_day.append((start_day, end_day))
            avg_length_day += end_day - start_day

            start_day_str = (
                str(start_day // 60)
                + "h"
                + (
                    str(start_day % 60)
                    if start_day % 60 >= 10
                    else "0" + str(start_day % 60)
                )
            )
            end_day_str = (
                str(end_day // 60)
                + "h"
                + (str(end_day % 60) if end_day % 60 >= 10 else "0" + str(end_day % 60))
            )

            print(f"Start of the day: {start_day_str}; End of the day: {end_day_str}")
        print("\n")

    print(f"Toal number of visited nodes: {nb_visits}")
    print(f"Overall total commuting time: {overall_total_commuting_time} minutes")
    print(f"Number of cars used: {nb_cars_used}")
    eq_co2_cars = eq_co2_car(overall_total_car_commuting_time)
    print(
        f"Overall total commuting time for cars: {overall_total_car_commuting_time} minutes ({eq_co2_cars} kgCO2)"
    )
    print(f"Number of bikes used: {nb_bikes_used}")
    eq_co2_bikes = eq_co2_elec_bike(overall_total_bike_commuting_time)
    print(
        f"Overall total commuting time for bikes: {overall_total_bike_commuting_time} minutes ({eq_co2_bikes} kgCO2)"
    )

    avg_length_day = avg_length_day / nb_inter_used
    avg_length_day_str = round(avg_length_day)
    avg_length_day_str = (
        str(avg_length_day_str // 60)
        + "h"
        + (
            str(avg_length_day_str % 60)
            if avg_length_day_str % 60 >= 10
            else "0" + str(avg_length_day_str % 60)
        )
    )
    print(f"Average length of a day of work: {avg_length_day_str}")

    dict_results = {
        "total_commuting_time": overall_total_commuting_time,
        "total_car_commuting_time": overall_total_car_commuting_time,
        "total_bike_commuting_time": overall_total_bike_commuting_time,
        "nb_inter_used": nb_inter_used,
        "nb_cars_used": nb_cars_used,
        "nb_bikes_used": nb_bikes_used,
        "avg_length_day": avg_length_day,
        "list_length_day": list_length_day,
        "eq_co2_cars": eq_co2_cars,
        "eq_co2_bikes": eq_co2_bikes,
        "nb_break_inf": nb_break_inf,
        "nb_break_sup": nb_break_sup,
        "nb_visits": nb_visits,
    }

    return dict_results

# 7. Running scenarios
